In [ ]:
# 3. RAGAS를 사용한 RAG 평가

from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from tools.rag.retriever.policy_pdf_retriever import PolicyPDFRetriever
import pandas as pd
import cohere
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# RAGAS 평가 메트릭 설명:
# - **Faithfulness (충실성)**: 생성된 답변이 검색된 문맥에 얼마나 기반하고 있는가? (환각 여부)
# - **LLMContextRecall (문맥 리콜)**: 검색된 문맥이 정답을 만드는 데 필요한 정보를 포함하는가? (검색 누락 여부)
# - **FactualCorrectness (사실 정확성)**: 생성된 답변이 실제 정답과 얼마나 일치하는가?

# LLM 설정
# 사용 가능한 모델: gpt-4o, gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo
model = ChatOpenAI(model="gpt-4o")
eval_llm = LangchainLLMWrapper(model)

# Retriever 초기화
retriever = PolicyPDFRetriever()

# 테스트 데이터 준비
test_data = [
    {
        "user_input": "2025년 6월 27일 정책에서 디딤돌 대출 일반 한도는 얼마인가?",
        "reference": "2.5억 원에서 2억 원으로 축소",
    },
    {
        "user_input": "10.15 대책에서 서울 규제지역은 어디인가?",
        "reference": "서울 25개구 전체",
    },
]

# RAGAS 평가용 샘플 생성
samples = []
co = cohere.Client(os.getenv("COHERE_API_KEY"))
for item in test_data:
    # 1. 질문으로 PDF 검색 (reranking을 위해 더 많이 검색)
    query = item["user_input"]
    docs = retriever.hybrid_search(query, k=20)

    # 2. Cohere Reranker 적용
    doc_texts = [doc.page_content for doc in docs]
    rerank_results = co.rerank(
        query=query,
        documents=doc_texts,
        top_n=6,  # 최종적으로 6개만 선택
        model="rerank-multilingual-v3.0",  # 한국어 지원 모델
    )

    # 3. Rerank된 순서대로 문서 재정렬
    reranked_docs = [docs[result.index] for result in rerank_results.results]

    # 4. 검색된 문맥 추출 (rerank된 문서 사용)
    retrieved_contexts = [doc.page_content for doc in reranked_docs]

    # 5. LLM으로 답변 생성
    context_text = "\n\n".join(retrieved_contexts)
    prompt = f"다음 문맥을 바탕으로 질문에 답하세요.\n\n문맥:\n{context_text}\n\n질문: {query}\n답변:"
    response = model.invoke(prompt).content

    # 6. RAGAS 샘플 생성
    sample = SingleTurnSample(
        user_input=query,
        retrieved_contexts=retrieved_contexts,
        response=response,
        reference=item["reference"],
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples=samples)

# RAGAS 평가 실행
result = evaluate(
    dataset=dataset,
    metrics=[
        Faithfulness(),
        LLMContextRecall(),
        FactualCorrectness(),
    ],
    llm=eval_llm,
)

# 결과 출력
print("RAGAS 평가 결과")
print("=" * 100)
print(result)

# DataFrame으로 상세 결과 확인
df = result.to_pandas()
print("\n상세 결과:")
print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_45564\2406965509.py:29: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  eval_llm = LangchainLLMWrapper(model)
C:\RAG_COMMANDER\src\tools\rag\vector_store.py:33: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation.
  _pgvector_cache[collection_name] = PGVector(


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

RAGAS 평가 결과
{'faithfulness': 1.0000, 'context_recall': 1.0000, 'factual_correctness(mode=f1)': 0.3350}

상세 결과:
                              user_input  \
0  2025년 6월 27일 정책에서 디딤돌 대출 일반 한도는 얼마인가?   
1              10.15 대책에서 서울 규제지역은 어디인가?   

                                  retrieved_contexts  \
0  [현행 개선 방안 시행 시기\n주담대\n총액한도 없음 수도권․규제지역6억원* ‘25...   
1  [ 토지거래허가구역 추가 지정\n□ (주요내용) 투기적 거래가 성행할 우려가 있는...   

                                            response           reference  \
0  2025년 6월 27일 정책에서 디딤돌 대출 일반 한도는 수도권 1.2억원, 지방 ...  2.5억 원에서 2억 원으로 축소   
1  10.15 대책에서 서울의 규제지역은 서울시 전역으로, 25개 자치구 전체가 포함됩...          서울 25개구 전체   

   faithfulness  context_recall  factual_correctness(mode=f1)  
0           1.0             1.0                          0.00  
1           1.0             1.0                          0.67  


In [ ]:
# 3. RAGAS를 사용한 RAG 평가

from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from tools.rag.retriever.policy_pdf_retriever import PolicyPDFRetriever
import pandas as pd
import cohere
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# -------------cohere
co = cohere.Client(os.getenv("COHERE_API_KEY"))


# -------------
# RAGAS 평가 메트릭 설명:
# - **Faithfulness (충실성)**: 생성된 답변이 검색된 문맥에 얼마나 기반하고 있는가? (환각 여부)
# - **LLMContextRecall (문맥 리콜)**: 검색된 문맥이 정답을 만드는 데 필요한 정보를 포함하는가? (검색 누락 여부)
# - **FactualCorrectness (사실 정확성)**: 생성된 답변이 실제 정답과 얼마나 일치하는가?

# LLM 설정
# 사용 가능한 모델: gpt-4o, gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo
model = ChatOpenAI(model="gpt-4o")
eval_llm = LangchainLLMWrapper(model)

# Retriever 초기화
retriever = PolicyPDFRetriever()

# 테스트 데이터 준비
test_data = [
    {
        "user_input": "2025년 6월 27일 정책에서 디딤돌 대출 일반 한도는 얼마인가?",
        "reference": "2.5억 원에서 2억 원으로 축소",
    },
    {
        "user_input": "10.15 대책에서 서울 규제지역은 어디인가?",
        "reference": "서울 25개구 전체",
    },
]

# RAGAS 평가용 샘플 생성
samples = []

for item in test_data:
    # 1. 질문으로 PDF 검색 (reranking을 위해 더 많이 검색)
    query = item["user_input"]
    docs = retriever.hybrid_search(query, k=20)

    # 2. Cohere Reranker 적용
    doc_texts = [doc.page_content for doc in docs]
    rerank_results = co.rerank(
        query=query,
        documents=doc_texts,
        top_n=6,  # 최종적으로 6개만 선택
        model="rerank-multilingual-v3.0",  # 한국어 지원 모델
    )

    # 3. Rerank된 순서대로 문서 재정렬
    reranked_docs = [docs[result.index] for result in rerank_results.results]

    # 4. 검색된 문맥 추출 (rerank된 문서 사용)
    retrieved_contexts = [doc.page_content for doc in reranked_docs]

    # 5. LLM으로 답변 생성
    context_text = "\n\n".join(retrieved_contexts)
    prompt = f"다음 문맥을 바탕으로 질문에 답하세요.\n\n문맥:\n{context_text}\n\n질문: {query}\n답변:"
    response = model.invoke(prompt).content

    # 6. RAGAS 샘플 생성
    sample = SingleTurnSample(
        user_input=query,
        retrieved_contexts=retrieved_contexts,
        response=response,
        reference=item["reference"],
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples=samples)

# RAGAS 평가 실행
result = evaluate(
    dataset=dataset,
    metrics=[
        Faithfulness(),
        LLMContextRecall(),
        FactualCorrectness(),
    ],
    llm=eval_llm,
)

# 결과 출력
print("RAGAS 평가 결과")
print("=" * 100)
print(result)

# DataFrame으로 상세 결과 확인
df = result.to_pandas()
print("\n상세 결과:")
print(df)

## 성능 비교: Reranker 적용 전 vs 후

아래 셀을 실행하면 Reranker를 사용하지 않은 경우와 사용한 경우의 성능을 비교할 수 있습니다.


In [ ]:
# Reranker 적용 전후 성능 비교

from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from tools.rag.retriever.policy_pdf_retriever import PolicyPDFRetriever
import pandas as pd
import cohere
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 초기화
co = cohere.Client(os.getenv("COHERE_API_KEY"))
model = ChatOpenAI(model="gpt-4o-mini")
eval_llm = LangchainLLMWrapper(model)
retriever = PolicyPDFRetriever()

# 테스트 데이터
test_data = [
    {
        "user_input": "2025년 6월 27일 정책에서 디딤돌 대출 일반 한도는 얼마인가?",
        "reference": "수도권 1.2억 원, 지방 0.8억 원",
    },
    {
        "user_input": "10.15 대책에서 서울 규제지역은 어디인가?",
        "reference": "서울 25개구 전체",
    },
    {
        "user_input": "생애최초 주택구입자의 LTV는 몇 퍼센트인가?",
        "reference": "70%",
    },
]


def run_evaluation(use_reranker=False, test_name=""):
    """평가 실행 함수"""
    samples = []

    for item in test_data:
        query = item["user_input"]

        if use_reranker:
            # Reranker 적용
            docs = retriever.hybrid_search(query, k=20)
            doc_texts = [doc.page_content for doc in docs]
            rerank_results = co.rerank(
                query=query,
                documents=doc_texts,
                top_n=6,
                model="rerank-multilingual-v3.0",
            )
            reranked_docs = [docs[result.index] for result in rerank_results.results]
            retrieved_contexts = [doc.page_content for doc in reranked_docs]
        else:
            # Reranker 미적용
            docs = retriever.hybrid_search(query, k=6)
            retrieved_contexts = [doc.page_content for doc in docs]

        # LLM 답변 생성
        context_text = "\n\n".join(retrieved_contexts)
        prompt = f"다음 문맥을 바탕으로 질문에 답하세요.\n\n문맥:\n{context_text}\n\n질문: {query}\n답변:"
        response = model.invoke(prompt).content

        # 샘플 생성
        sample = SingleTurnSample(
            user_input=query,
            retrieved_contexts=retrieved_contexts,
            response=response,
            reference=item["reference"],
        )
        samples.append(sample)

    # 평가 실행
    dataset = EvaluationDataset(samples=samples)
    result = evaluate(
        dataset=dataset,
        metrics=[
            Faithfulness(),
            LLMContextRecall(),
            FactualCorrectness(),
        ],
        llm=eval_llm,
    )

    print(f"\n{'='*80}")
    print(f"📊 {test_name}")
    print(f"{'='*80}")
    print(result)
    print(f"\n상세 결과:")
    df = result.to_pandas()
    print(
        df[
            [
                "user_input",
                "faithfulness",
                "context_recall",
                "factual_correctness(mode=f1)",
            ]
        ]
    )

    return result


# 실행
print("🔵 Reranker 미적용 테스트 시작...")
result_without_reranker = run_evaluation(
    use_reranker=False, test_name="Reranker 미적용"
)

print("\n🟢 Reranker 적용 테스트 시작...")
result_with_reranker = run_evaluation(use_reranker=True, test_name="Reranker 적용")

# 비교 요약
print(f"\n{'='*80}")
print("📈 성능 비교 요약")
print(f"{'='*80}")

metrics_comparison = {
    "Faithfulness": [
        result_without_reranker["faithfulness"],
        result_with_reranker["faithfulness"],
    ],
    "Context Recall": [
        result_without_reranker["context_recall"],
        result_with_reranker["context_recall"],
    ],
    "Factual Correctness": [
        result_without_reranker["factual_correctness(mode=f1)"],
        result_with_reranker["factual_correctness(mode=f1)"],
    ],
}

comparison_df = pd.DataFrame(
    metrics_comparison, index=["Reranker 미적용", "Reranker 적용"]
)

print("\n성능 비교표:")
print(comparison_df)

# 개선율 계산
print("\n개선율:")
for metric in metrics_comparison.keys():
    before = metrics_comparison[metric][0]
    after = metrics_comparison[metric][1]
    improvement = ((after - before) / before * 100) if before > 0 else 0
    print(f"{metric}: {improvement:+.1f}%")